# an examination of the creditcard (ULB) dataset and appliance of SMOTE method

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

In [3]:
df = pd.read_csv('data/creditcard.csv')
print(df.head())
ds = df.to_numpy()
print(ds.shape)

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [4]:
print(df['Class'].value_counts())

y = ds[:, -1] # for last column
x = ds[:, :-1] # for all but last column

print(y.shape)
print(len(y[y == 0]))
print(len(y[y == 1]))
print(x.shape)


print(type(y[0]))

0    284315
1       492
Name: Class, dtype: int64
(284807,)
284315
492
(284807, 30)
<class 'numpy.float64'>


Der Datensatz ist stark unbalanciert. Wie man oben sehen kann sind nur 492 Frauds in dem Datensatz und 284.315 non-Frauds. Ohne Preprocessing und mit so einer starken Inbalance darf man einige Modelle nicht anwenden (z.B. neuronale Netze)

Der Datensatz ist stark unbalanciert. Wie man oben sehen kann sind nur 492 Frauds in dem Datensatz und 284.315 non-Frauds. Ohne Preprocessing und mit so einer starken Inbalance darf man einige Modelle nicht anwenden (z.B. neuronale Netze)

folgende Datensätze werden nun erzeugt:
ds_major_down.csv - Datensatz bei dem die Majority Class gedownsampelt wird (*** - macht eigentlich wenig Sinn)
ds_minor_up.csv - Datensatz bei dem die Minority Class künstlich vergrößert wird.
[!] wir werden SMOTE anwenden, muss noch recherchiert werden wie gut diese Mothode ist, aber scheint sehr gängig zu sein sieh: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [5]:
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
print(imblearn.__version__)

print(x.shape)

# define pipeline
over = SMOTE(sampling_strategy=0.02, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.7)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X_sampled, y_sampled = pipeline.fit_resample(x, y)

print('class 0 (orig): ',len(y[y == 0]))
print('class 1 (orig): ', len(y[y == 1]))

print('class 0: ', len(y_sampled[y_sampled == 0]))
print('class 1: ', len(y_sampled[y_sampled == 1]))

print(x.shape)

0.9.1
(284807, 30)
class 0 (orig):  284315
class 1 (orig):  492
class 0:  8122
class 1:  5686
(284807, 30)


speichere den neuen Datensatz mit

class 0:  8122
class 1:  5686


In [6]:
new_dataset = pd.DataFrame(data=np.column_stack((X_sampled, y_sampled)), columns=df.columns)
new_dataset.to_csv('data/creditcard_newds.csv', index=False)

In [7]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,82450.0,1.314539,0.590643,-0.666593,0.716564,0.301978,-1.125467,0.388881,-0.288390,-0.132137,...,-0.170307,-0.429655,-0.141341,-0.200195,0.639491,0.399476,-0.034321,0.031692,0.76,0.0
1,50554.0,-0.798672,1.185093,0.904547,0.694584,0.219041,-0.319295,0.495236,0.139269,-0.760214,...,0.202287,0.578699,-0.092245,0.013723,-0.246466,-0.380057,-0.396030,-0.112901,4.18,0.0
2,55125.0,-0.391128,-0.245540,1.122074,-1.308725,-0.639891,0.008678,-0.701304,-0.027315,-2.628854,...,-0.133485,0.117403,-0.191748,-0.488642,-0.309774,0.008100,0.163716,0.239582,15.00,0.0
3,116572.0,-0.060302,1.065093,-0.987421,-0.029567,0.176376,-1.348539,0.775644,0.134843,-0.149734,...,0.355576,0.907570,-0.018454,-0.126269,-0.339923,-0.150285,-0.023634,0.042330,57.00,0.0
4,90434.0,1.848433,0.373364,0.269272,3.866438,0.088062,0.970447,-0.721945,0.235983,0.683491,...,0.103563,0.620954,0.197077,0.692392,-0.206530,-0.021328,-0.019823,-0.042682,0.00,0.0


In [8]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
